In [1]:
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-12-23 22:21:00--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-12-23 22:21:00--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv.1’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.004s  

2024-12-23 22:21:01 (94.8 MB/s) - ‘jamb_exam_re

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [3]:
df = pd.read_csv('jamb_exam_results.csv')
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [5]:
df = df.drop(columns=['student_id'])
df = df.fillna(0)

In [6]:
df_full_train, df_test = train_test_split(df, test_size= 0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size= 0.25, random_state=1)

In [36]:
dv = DictVectorizer(sparse=False)
def separate_x_y(df):    
    y = df['assignments_completed']
    x = df.drop(columns=['assignments_completed'])
    train_dicts = x.to_dict(orient='records')
    x = dv.fit_transform(train_dicts)
    return x, y
x_train, y_train = separate_x_y(df_train)
x_val, y_val = separate_x_y(df_val)
x_test, y_test = separate_x_y(df_test)

## Question 1

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
dt = DecisionTreeClassifier(max_depth = 1)
dt.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=1)

In [38]:
y_pred = dt.predict_proba(x_train)
roc_auc_score(y_train, y_pred, multi_class='ovr', average='macro')

np.float64(0.7173982344072003)

In [39]:
y_pred = dt.predict_proba(x_val)
roc_auc_score(y_val, y_pred, multi_class='ovr', average='macro')


np.float64(0.6943862871231928)

In [40]:
from sklearn.tree import export_text
print(export_text(dt, feature_names=dv.get_feature_names_out()))

|--- study_hours_per_week <= 19.50
|   |--- class: 1
|--- study_hours_per_week >  19.50
|   |--- class: 2



#### study_hours_per_week